In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import ast
import matplotlib.pyplot as plt
from google.colab import drive
import os
import logging
import pickle

In [2]:
# Initialisation
# Mount Google Drive
drive.mount('/content/drive')

# Define paths
data_path = '/content/drive/MyDrive/smai_project/dataset/dataset20/'
output_path = '/content/drive/MyDrive/smai_project/output_graphs/'
model_path = '/content/drive/MyDrive/smai_project/model/model20/'
log_file_path = os.path.join(output_path, 'train.log')

# Create output and model directories if they don't exist
for path in [output_path, model_path]:
  if not os.path.exists(path):
    os.makedirs(path)

# Set up logging
logging.basicConfig( level=logging.INFO, format='%(asctime)s - %(message)s', handlers=[ logging.FileHandler(log_file_path), logging.StreamHandler() ] )

Mounted at /content/drive


In [3]:
# Feature Extraction Function
def extract_sequence_features(df):
    """Extract features from ball and player sequences."""
    def get_shooter_position(ball_seq, players_seq, personId):
        shooter_x, shooter_y = None, None
        if players_seq and isinstance(players_seq, list) and len(players_seq) > 0:
            first_frame = players_seq[0] if isinstance(players_seq[0], list) else players_seq
            if first_frame and isinstance(first_frame, list):
                for player in first_frame:
                    if len(player) != 5:
                        continue
                    _, player_id, x, y, _ = player
                    if player_id == personId:
                        shooter_x, shooter_y = x, y
                        break

        if shooter_x is None or shooter_y is None:
            if not ball_seq or not isinstance(ball_seq, list) or len(ball_seq) == 0:
                return 0, 0
            first_pos = ball_seq[0]
            if not isinstance(first_pos, list) or len(first_pos) != 3:
                return 0, 0
            shooter_x, shooter_y, _ = first_pos
        return shooter_x, shooter_y

    df['shooter_x'], df['shooter_y'] = zip(*df.apply(
        lambda row: get_shooter_position(row['ball_seq'], row['players_seq'], row['personId']), axis=1
    ))

    df['initial_height'] = df['ball_seq'].apply(lambda seq: seq[0][2] if len(seq) > 0 and len(seq[0]) == 3 else 0)
    df['max_height'] = df['ball_seq'].apply(lambda seq: max([point[2] for point in seq]) if len(seq) > 0 and all(len(point) == 3 for point in seq) else 0)
    df['traj_length'] = df['ball_seq'].apply(len)
    df['shotDistance'] = df.apply(
        lambda row: np.sqrt((row['shooter_x'] - row['basket_x'])**2 + (row['shooter_y'] - row['basket_y'])**2), axis=1
    )
    df['shotDistance'] = df['shotDistance'].replace(0, 1)
    df['traj_curvature'] = df['max_height'] / df['shotDistance']

    df['release_angle'] = df.apply(
        lambda row: np.arctan2(row['shooter_y'] - row['basket_y'], row['shooter_x'] - row['basket_x']), axis=1
    )

    def calculate_defender_proximity(shooter_x, shooter_y, players_seq):
        min_distance = float('inf')
        shooter_team_id = None

        if not players_seq or not isinstance(players_seq, list) or len(players_seq) == 0:
            return 0

        first_frame = players_seq[0] if isinstance(players_seq[0], list) else players_seq
        if not first_frame or not isinstance(first_frame, list):
            return 0

        closest_player_distance = float('inf')
        for player in first_frame:
            if len(player) != 5:
                continue
            team_id, _, x, y, _ = player
            distance = np.sqrt((shooter_x - x)**2 + (shooter_y - y)**2)
            if distance < closest_player_distance:
                closest_player_distance = distance
                shooter_team_id = team_id

        if shooter_team_id is None:
            logging.warning(f"Could not determine shooter team for position ({shooter_x}, {shooter_y})")
            return 0

        for player in first_frame:
            if len(player) != 5:
                continue
            team_id, _, x, y, _ = player
            if team_id == shooter_team_id:
                continue
            distance = np.sqrt((shooter_x - x)**2 + (shooter_y - y)**2)
            min_distance = min(min_distance, distance)
        return min_distance if min_distance != float('inf') else 0

    df['defender_proximity'] = df.apply(
        lambda row: calculate_defender_proximity(row['shooter_x'], row['shooter_y'], row['players_seq']), axis=1
    )

    def extract_player_positions(players_seq, shooter_x, shooter_y):
        shooter_team_id = None
        teammates = []
        defenders = []

        if not players_seq or not isinstance(players_seq, list) or len(players_seq) == 0:
            return [0, 0, 0, 0]

        first_frame = players_seq[0] if isinstance(players_seq[0], list) else players_seq
        if not first_frame or not isinstance(first_frame, list):
            return [0, 0, 0, 0]

        closest_player_distance = float('inf')
        for player in first_frame:
            if len(player) != 5:
                continue
            team_id, _, x, y, _ = player
            distance = np.sqrt((shooter_x - x)**2 + (shooter_y - y)**2)
            if distance < closest_player_distance:
                closest_player_distance = distance
                shooter_team_id = team_id

        if shooter_team_id is None:
            logging.warning(f"Could not determine shooter team for position ({shooter_x}, {shooter_y})")
            return [0, 0, 0, 0]

        for player in first_frame:
            if len(player) != 5:
                continue
            team_id, _, x, y, _ = player
            if team_id == shooter_team_id:
                teammates.append([x, y])
            else:
                defenders.append([x, y])

        closest_teammate = [0, 0]
        if teammates:
            teammate_distances = [(np.sqrt((shooter_x - tx)**2 + (shooter_y - ty)**2), tx, ty) for tx, ty in teammates]
            teammate_distances.sort()
            closest_teammate = [teammate_distances[1][1], teammate_distances[1][2]] if len(teammate_distances) > 1 else teammate_distances[0][1:3]

        closest_defender = [0, 0]
        if defenders:
            defender_distances = [(np.sqrt((shooter_x - dx)**2 + (shooter_y - dy)**2), dx, dy) for dx, dy in defenders]
            defender_distances.sort()
            closest_defender = [defender_distances[0][1], defender_distances[0][2]]

        return closest_teammate + closest_defender

    df[['teammate_x', 'teammate_y', 'defender_x', 'defender_y']] = df.apply(
        lambda row: extract_player_positions(row['players_seq'], row['shooter_x'], row['shooter_y']), axis=1, result_type='expand'
    )

    def extract_ball_dynamics(ball_seq):
        if not ball_seq or len(ball_seq) < 3:
            return [0, 0, 0, 0, 0, 0]

        velocities = []
        for i in range(1, len(ball_seq)):
            if len(ball_seq[i]) != 3 or len(ball_seq[i-1]) != 3:
                continue
            dx = ball_seq[i][0] - ball_seq[i-1][0]
            dy = ball_seq[i][1] - ball_seq[i-1][1]
            dz = ball_seq[i][2] - ball_seq[i-1][2]
            velocities.append([dx, dy, dz])

        if not velocities:
            return [0, 0, 0, 0, 0, 0]

        accelerations = []
        for i in range(1, len(velocities)):
            ax = velocities[i][0] - velocities[i-1][0]
            ay = velocities[i][1] - velocities[i-1][1]
            az = velocities[i][2] - velocities[i-1][2]
            accelerations.append([ax, ay, az])

        if not accelerations:
            avg_vel_x = np.mean([v[0] for v in velocities])
            avg_vel_y = np.mean([v[1] for v in velocities])
            avg_vel_z = np.mean([v[2] for v in velocities])
            return [avg_vel_x, avg_vel_y, avg_vel_z, 0, 0, 0]

        avg_vel_x = np.mean([v[0] for v in velocities])
        avg_vel_y = np.mean([v[1] for v in velocities])
        avg_vel_z = np.mean([v[2] for v in velocities])
        avg_acc_x = np.mean([a[0] for a in accelerations])
        avg_acc_y = np.mean([a[1] for a in accelerations])
        avg_acc_z = np.mean([a[2] for a in accelerations])

        return [avg_vel_x, avg_vel_y, avg_vel_z, avg_acc_x, avg_acc_y, avg_acc_z]

    df[['avg_vel_x', 'avg_vel_y', 'avg_vel_z', 'avg_acc_x', 'avg_acc_y', 'avg_acc_z']] = df['ball_seq'].apply(
        lambda seq: extract_ball_dynamics(seq)
    ).apply(pd.Series)

    def extract_shot_arc(ball_seq, basket_x, basket_y):
        if not ball_seq or len(ball_seq) < 3:
            return [0, 0]

        try:
            release_x, release_y, release_z = ball_seq[0]
            peak_idx = max(range(len(ball_seq)), key=lambda i: ball_seq[i][2] if len(ball_seq[i]) == 3 else 0)
            peak_x, peak_y, peak_z = ball_seq[peak_idx]

            last_valid_idx = -1
            for i in range(len(ball_seq)-1, 0, -1):
                if len(ball_seq[i]) == 3:
                    last_valid_idx = i
                    break

            if last_valid_idx <= 0:
                return [0, 0]

            p2 = ball_seq[last_valid_idx]
            p1 = ball_seq[max(0, last_valid_idx-1)]

            if len(p1) != 3 or len(p2) != 3:
                return [0, 0]

            horizontal_dist = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
            entry_angle = 90 if horizontal_dist == 0 else np.degrees(np.arctan2(p2[2] - p1[2], horizontal_dist))

            total_dist = np.sqrt((basket_x - release_x)**2 + (basket_y - release_y)**2)
            arc_height_ratio = 0 if total_dist == 0 else peak_z / total_dist

            return [entry_angle, arc_height_ratio]
        except Exception as e:
            logging.error(f"Error extracting shot arc: {e}")
            return [0, 0]

    df[['entry_angle', 'arc_height_ratio']] = df.apply(
        lambda row: extract_shot_arc(row['ball_seq'], row['basket_x'], row['basket_y']),
        axis=1, result_type='expand'
    )

    return df

# Sequence Processing Function
def process_ball_sequences(df):
    """Process ball sequences for LSTM input."""
    sequences = []
    labels = []

    for _, row in df.iterrows():
        ball_seq = row['ball_seq']
        result = row['shotResult']

        if len(ball_seq) > 37:
            ball_seq = ball_seq[:37]
        elif len(ball_seq) < 37:
            last_pos = ball_seq[-1] if ball_seq else [0, 0, 0]
            while len(ball_seq) < 37:
                ball_seq.append(last_pos)

        sequence = []
        for moment in ball_seq:
            if isinstance(moment, list) and len(moment) == 3:
                sequence.append(moment)
            else:
                sequence.append([0, 0, 0])

        sequences.append(sequence)
        labels.append(result)

    return np.array(sequences), np.array(labels)

In [4]:
# Load and preprocess data
print("Loading datasets...")
train_df = pd.read_csv(data_path + 'train.csv')
val_df = pd.read_csv(data_path + 'val.csv')
test_df = pd.read_csv(data_path + 'test.csv')

# Parse sequences
for df in [train_df, val_df, test_df]:
    df['players_seq'] = df['players_seq'].apply(ast.literal_eval)
    df['ball_seq'] = df['ball_seq'].apply(ast.literal_eval)

# Filter shots and encode labels
for df in [train_df, val_df, test_df]:
    df.drop(df[~df['shotResult'].isin(['Made Shot', 'Missed Shot'])].index, inplace=True)
    df['shotResult'] = df['shotResult'].map({'Made Shot': 1, 'Missed Shot': 0})

# Extract features
print("Extracting features...")
train_df = extract_sequence_features(train_df)
val_df = extract_sequence_features(val_df)
test_df = extract_sequence_features(test_df)

# Define static features
static_features = [
    'shooter_x', 'shooter_y', 'release_angle', 'initial_height', 'max_height',
    'traj_length', 'traj_curvature', 'defender_proximity',
    'teammate_x', 'teammate_y', 'defender_x', 'defender_y',
    'avg_vel_x', 'avg_vel_y', 'avg_vel_z', 'avg_acc_x', 'avg_acc_y', 'avg_acc_z',
    'entry_angle', 'arc_height_ratio'
]

# Clean up NaN or inf values
for df in [train_df, val_df, test_df]:
    df[static_features] = df[static_features].replace([np.inf, -np.inf], 0).fillna(0)

# Process ball sequences
print("Processing ball sequences...")
X_train_ball, y_train = process_ball_sequences(train_df)
X_val_ball, y_val = process_ball_sequences(val_df)
X_test_ball, y_test = process_ball_sequences(test_df)

# Reshape for LSTM
X_train_ball = X_train_ball.reshape(X_train_ball.shape[0], 37, 3)
X_val_ball = X_val_ball.reshape(X_val_ball.shape[0], 37, 3)
X_test_ball = X_test_ball.reshape(X_test_ball.shape[0], 37, 3)

# Normalize ball sequences
scaler_ball = MinMaxScaler()
X_train_ball_2d = X_train_ball.reshape(-1, 3)
X_train_ball_2d = scaler_ball.fit_transform(X_train_ball_2d)
X_train_ball = X_train_ball_2d.reshape(X_train_ball.shape)

X_val_ball_2d = X_val_ball.reshape(-1, 3)
X_val_ball_2d = scaler_ball.transform(X_val_ball_2d)
X_val_ball = X_val_ball_2d.reshape(X_val_ball.shape)

X_test_ball_2d = X_test_ball.reshape(-1, 3)
X_test_ball_2d = scaler_ball.transform(X_test_ball_2d)
X_test_ball = X_test_ball_2d.reshape(X_test_ball.shape)

# Process static features
X_train_static = train_df[static_features].values
X_val_static = val_df[static_features].values
X_test_static = test_df[static_features].values

# Normalize static features
scaler_static = MinMaxScaler()
X_train_static = scaler_static.fit_transform(X_train_static)
X_val_static = scaler_static.transform(X_val_static)
X_test_static = scaler_static.transform(X_test_static)

Loading datasets...
Extracting features...
Processing ball sequences...


In [5]:
# Define PyTorch Dataset
class ShotDataset(Dataset):
    """Custom Dataset for basketball shot data."""
    def __init__(self, ball_sequences, static_features, labels):
        self.ball_sequences = torch.tensor(ball_sequences, dtype=torch.float32)
        self.static_features = torch.tensor(static_features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.ball_sequences[idx], self.static_features[idx], self.labels[idx]

# Define PyTorch Model
class ShotPredictor(nn.Module):
    """PyTorch model for basketball shot prediction."""
    def __init__(self, ball_input_size=3, static_input_size=20, hidden_size=64):
        super(ShotPredictor, self).__init__()
        self.lstm1 = nn.LSTM(ball_input_size, hidden_size, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size // 2, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)

        self.static_fc1 = nn.Linear(static_input_size, 32)
        self.static_relu = nn.ReLU()
        self.static_dropout = nn.Dropout(0.2)

        self.fc_combined = nn.Linear(hidden_size // 2 + 32, 32)
        self.relu_combined = nn.ReLU()
        self.dropout_combined = nn.Dropout(0.2)
        self.fc_out = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, ball_seq, static_features):
        # Ball sequence processing
        lstm_out, _ = self.lstm1(ball_seq)
        lstm_out = self.dropout1(lstm_out)
        lstm_out, _ = self.lstm2(lstm_out)
        lstm_out = self.dropout2(lstm_out)
        lstm_out = lstm_out[:, -1, :]  # Take the last time step

        # Static features processing
        static_out = self.static_fc1(static_features)
        static_out = self.static_relu(static_out)
        static_out = self.static_dropout(static_out)

        # Combine
        combined = torch.cat((lstm_out, static_out), dim=1)
        out = self.fc_combined(combined)
        out = self.relu_combined(out)
        out = self.dropout_combined(out)
        out = self.fc_out(out)
        out = self.sigmoid(out)
        return out

# Create datasets
train_dataset = ShotDataset(X_train_ball, X_train_static, y_train)
val_dataset = ShotDataset(X_val_ball, X_val_static, y_val)
test_dataset = ShotDataset(X_test_ball, X_test_static, y_test)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# Initialize model, loss, and optimizer
print("Building model...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ShotPredictor(ball_input_size=3, static_input_size=len(static_features)).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
print("Training model...")
num_epochs = 20
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for ball_seq, static_features, labels in train_loader:
        ball_seq, static_features, labels = ball_seq.to(device), static_features.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(ball_seq, static_features).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * ball_seq.size(0)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for ball_seq, static_features, labels in val_loader:
            ball_seq, static_features, labels = ball_seq.to(device), static_features.to(device), labels.to(device)
            outputs = model(ball_seq, static_features).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item() * ball_seq.size(0)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_dataset)
    val_acc = correct / total
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

# Plot and save training metrics
plt.figure(figsize=(10, 5))
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Training and Validation Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_path, 'accuracy_over_epochs.png'))
plt.close()

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_path, 'loss_over_epochs.png'))
plt.close()

# Save model and scalers
print("Saving model and scalers...")
torch.save(model.state_dict(), os.path.join(model_path, 'shot_model.pth'))
with open(os.path.join(model_path, 'scaler_ball.pkl'), 'wb') as f:
    pickle.dump(scaler_ball, f)
with open(os.path.join(model_path, 'scaler_static.pkl'), 'wb') as f:
    pickle.dump(scaler_static, f)

print("Training completed!")

Building model...
Training model...
Epoch [1/20], Train Loss: 0.6652, Train Acc: 0.5905, Val Loss: 0.6244, Val Acc: 0.6759
Epoch [2/20], Train Loss: 0.6001, Train Acc: 0.6957, Val Loss: 0.5799, Val Acc: 0.7101
Epoch [3/20], Train Loss: 0.5548, Train Acc: 0.7268, Val Loss: 0.5466, Val Acc: 0.7357
Epoch [4/20], Train Loss: 0.5334, Train Acc: 0.7378, Val Loss: 0.5759, Val Acc: 0.6939
Epoch [5/20], Train Loss: 0.5306, Train Acc: 0.7342, Val Loss: 0.4986, Val Acc: 0.7519
Epoch [6/20], Train Loss: 0.5102, Train Acc: 0.7474, Val Loss: 0.4997, Val Acc: 0.7576
Epoch [7/20], Train Loss: 0.5166, Train Acc: 0.7433, Val Loss: 0.5069, Val Acc: 0.7481
Epoch [8/20], Train Loss: 0.5015, Train Acc: 0.7543, Val Loss: 0.5053, Val Acc: 0.7519
Epoch [9/20], Train Loss: 0.4936, Train Acc: 0.7541, Val Loss: 0.5173, Val Acc: 0.7367
Epoch [10/20], Train Loss: 0.4900, Train Acc: 0.7590, Val Loss: 0.4557, Val Acc: 0.7890
Epoch [11/20], Train Loss: 0.4591, Train Acc: 0.7839, Val Loss: 0.4152, Val Acc: 0.8089
Epoch